# Load data

In [1]:
import json

input_file = "./batch.json"
with open(input_file, 'r') as f:
    data = json.load(f)

In [2]:
import json
from statistics import mean

# Load the JSON data from the file
file_path = 'batch.json'
with open(file_path, 'r') as file:
    data = json.load(file)

# Define the list of candidate models and judge models
candidate_model = ["llama_vid", "gpt4v", "video_chatgpt", "mplug_owl_Video", "video_llava"]
# judge_model = ['video_llava', 'llama_vid', 'gpt4v', 'internvl2']  # Assuming 'mplug_owl_Video' is not a judge
judge_model = ['gpt4v', 'internvl2']  # Assuming 'mplug_owl_Video' is not a judge

# Initialize the data structure for storing scores
dataset2data = {}

# Function to extract ratings from JSON-formatted strings
import re
import json

def extract_rating(rating_string):
    """
    Extracts the rating number from a string using regex to handle different formats.
    
    Args:
    rating_string (str): The input string containing the rating.
    
    Returns:
    int: The extracted rating number or None if not found.
    """
    # Regex pattern to find rating in different formats
    # patterns = [
    #     r"rating['\"]?\s*:\s*['\"]?(\d+)",  # Matches JSON-like formats with 'rating' key
    #     r"^(\d+)\s*\(",                     # Matches formats like "4 (Excellent)"
    #     r"^(\d+)$",                         # Matches plain number formats like "4"
    # ]
    patterns = [
        r"rating['\"]?\s*:\s*['\"]?(\d+)",   # Matches JSON-like formats with 'rating' key
        r"rating['\"]?\s*:\s*['\"]?\[\[(\d+)\]\]",  # Matches JSON-like formats with rating enclosed in double brackets
        r"rating['\"]?\s*:\s*['\"]?\[(\d+)\]",  # Matches JSON-like formats with rating enclosed in single brackets
        r"^(\d+)\s*\(",                      # Matches formats like "4 (Excellent)"
        r"^(\d+)$",                          # Matches plain number formats like "4"
    ]

    for pattern in patterns:
        match = re.search(pattern, rating_string)
        if match:
            return int(match.group(1))
    
    return None

# Iterate through each dataset and each sample, collecting ratings
for dataset, samples in data.items():
    results = {judge: {candidate: [] for candidate in candidate_model} for judge in judge_model}
    for sample_id, sample_data in samples.items():
        for judge in judge_model:
            if judge in sample_data:
                rating_info = sample_data[judge]
                for candidate, rating_str in rating_info.items():
                    rating = extract_rating(rating_str)
                    if rating is not None:
                        results[judge][candidate].append(rating)
                    else:
                        print(f'rating_str:{rating_str}')
                        print(f'rating:{rating}')
                        results[judge][candidate].append(0)
    dataset2data[dataset] = results

    # # Calculate average ratings for each candidate model under each judge for the dataset
    # for judge, candidates in results.items():
    #     for candidate, ratings in candidates.items():
    #         if ratings:
    #             average = round(mean(ratings), 2)
    #         else:
    #             average = None
    #         if dataset not in dataset2data:
    #             dataset2data[dataset] = {}
    #         if judge not in dataset2data[dataset]:
    #             dataset2data[dataset][judge] = {}
    #             dataset2data[dataset][judge][candidate] = results[judge][candidate]
                


rating_str:
rating:None


In [3]:
import json
# gpt_debate score
input_file = "../logs_phase1_processed/batch_with_score.json"
with open(input_file, 'r') as f:
    data = json.load(f)

In [4]:
for dataset, samples in data.items():
    results = {'gpt_debate': {candidate: [] for candidate in candidate_model}}
    for sample_id, sample_data in samples.items():
        for candidate in candidate_model:
            rating = sample_data[candidate][1]
            results['gpt_debate'][candidate].append(rating)
    dataset2data[dataset]['gpt_debate'] = results['gpt_debate']

# Get statistics of judge

In [5]:
missing_match = 0
judge_model_all = judge_model + ['gpt_debate']
judge2statistics = {judge: {i:0 for i in range(6)} for judge in judge_model_all}
for dataset, data in dataset2data.items():
    for judge in judge_model_all:
        for candidate in candidate_model:
            for rating in dataset2data[dataset][judge][candidate]:
                if rating:
                    judge2statistics[judge][rating] += 1
                else:
                    missing_match += 1
missing_match

2

In [6]:
dataset2data[dataset][judge][candidate]
candidate

'video_llava'

In [7]:
judge_model_all = judge_model + ['gpt_debate']
candidate2statistics = {dataset: {candidate: {judge:[] for judge in judge_model_all} for candidate in candidate_model} for dataset in dataset2data}
for dataset, data in dataset2data.items():
    for judge in judge_model_all:
        for candidate in candidate_model:
            for rating in dataset2data[dataset][judge][candidate]:
                candidate2statistics[dataset][candidate][judge].append(rating)
            candidate2statistics[dataset][candidate][judge] = sum(candidate2statistics[dataset][candidate][judge]) / len(candidate2statistics[dataset][candidate][judge])
candidate2statistics

{'cvrr_continuity_and_object_instance_count': {'llama_vid': {'gpt4v': 2.7344632768361583,
   'internvl2': 3.2542372881355934,
   'gpt_debate': 1.9435028248587571},
  'gpt4v': {'gpt4v': 3.056497175141243,
   'internvl2': 3.310734463276836,
   'gpt_debate': 2.4745762711864407},
  'video_chatgpt': {'gpt4v': 2.531073446327684,
   'internvl2': 2.977401129943503,
   'gpt_debate': 2.135593220338983},
  'mplug_owl_Video': {'gpt4v': 2.6045197740112993,
   'internvl2': 3.310734463276836,
   'gpt_debate': 1.8531073446327684},
  'video_llava': {'gpt4v': 2.672316384180791,
   'internvl2': 3.310734463276836,
   'gpt_debate': 1.9661016949152543}},
 'cvrr_fine_grained_action_understanding': {'llama_vid': {'gpt4v': 3.0478260869565217,
   'internvl2': 3.5434782608695654,
   'gpt_debate': 1.934782608695652},
  'gpt4v': {'gpt4v': 3.4391304347826086,
   'internvl2': 3.5521739130434784,
   'gpt_debate': 3.082608695652174},
  'video_chatgpt': {'gpt4v': 2.9608695652173913,
   'internvl2': 3.391304347826087,
 

# Naive Assembly

# Agreement

In [8]:
from sklearn.metrics import cohen_kappa_score

def get_agreement_with_baseline(dataset2data, dataset, judge, candidate):
    ratings1 = dataset2data[dataset][judge][candidate]
    ratings2 = dataset2data[dataset]['gpt_debate'][candidate]
    # print(f"ratings1:{ratings1}")
    # print(f"ratings2:{ratings2}")
    # Calculate Weighted Kappa with linear weights
    # kappa = cohen_kappa_score(ratings1, ratings2)
    # print("Kappa:", kappa)

    # # Calculate Weighted Kappa with linear weights
    # weighted_kappa_linear = cohen_kappa_score(ratings1, ratings2, weights='linear')
    # print("Weighted Kappa (Linear):", weighted_kappa_linear)

    # Calculate Weighted Kappa with quadratic weights
    weighted_kappa_quadratic = cohen_kappa_score(ratings1, ratings2, weights='quadratic')
    # print("Weighted Kappa (Quadratic):", weighted_kappa_quadratic)
    return weighted_kappa_quadratic

dataset2agreement = {}
for dataset, data in dataset2data.items():
    dataset2agreement[dataset] = {}
    for judge in judge_model:
        dataset2agreement[dataset][judge] = {}
        judge_total = 0
        for candidate in candidate_model:
            dataset2agreement[dataset][judge][candidate] = get_agreement_with_baseline(dataset2data, dataset, judge, candidate)
            judge_total += dataset2agreement[dataset][judge][candidate]
        dataset2agreement[dataset][judge]['average'] = judge_total / len(dataset2agreement[dataset][judge])

In [9]:
dataset2agreement

{'cvrr_continuity_and_object_instance_count': {'gpt4v': {'llama_vid': 0.1475962134678186,
   'gpt4v': 0.40171728079648117,
   'video_chatgpt': 0.28344166951761884,
   'mplug_owl_Video': 0.10580744178398194,
   'video_llava': 0.14180061676172817,
   'average': 0.21607264446552574},
  'internvl2': {'llama_vid': 0.02868342172008309,
   'gpt4v': 0.24977019711980386,
   'video_chatgpt': 0.08700480409434375,
   'mplug_owl_Video': 0.0046608237499367045,
   'video_llava': 0.04035856250506997,
   'average': 0.08209556183784747}},
 'cvrr_fine_grained_action_understanding': {'gpt4v': {'llama_vid': 0.2837257585051668,
   'gpt4v': 0.2641869533577734,
   'video_chatgpt': 0.3451393537246684,
   'mplug_owl_Video': 0.3768800827473643,
   'video_llava': 0.40361982463543855,
   'average': 0.3347103945940823},
  'internvl2': {'llama_vid': 0.14817963407872858,
   'gpt4v': 0.19636830472358613,
   'video_chatgpt': 0.13495183044315995,
   'mplug_owl_Video': 0.16215470036964896,
   'video_llava': 0.17253453384

In [10]:
dataset2average = {}
judge_model.append('gpt_debate')
for dataset, data in dataset2data.items():
    dataset2average[dataset] = {}
    for judge in judge_model:
        dataset2average[dataset][judge] = {}
        judge_total = 0
        for candidate in candidate_model:
            judge_total += sum(dataset2data[dataset][judge][candidate]) / len(dataset2data[dataset][judge][candidate])
        dataset2average[dataset][judge] = judge_total / len(candidate_model)

In [11]:
dataset2average

{'cvrr_continuity_and_object_instance_count': {'gpt4v': 2.719774011299435,
  'internvl2': 3.232768361581921,
  'gpt_debate': 2.074576271186441},
 'cvrr_fine_grained_action_understanding': {'gpt4v': 3.104347826086957,
  'internvl2': 3.485217391304348,
  'gpt_debate': 2.223478260869565},
 'cvrr_interpretation_of_social_context': {'gpt4v': 2.6,
  'internvl2': 3.164285714285714,
  'gpt_debate': 2.137142857142857},
 'cvrr_interpretation_of_visual_context': {'gpt4v': 2.9362637362637365,
  'internvl2': 3.2051282051282053,
  'gpt_debate': 2.183882783882784},
 'cvrr_multiple_actions_in_a_single_video': {'gpt4v': 2.687421383647799,
  'internvl2': 3.125157232704402,
  'gpt_debate': 1.89937106918239},
 'cvrr_non_existent_actions_with_existent_scene_depictions': {'gpt4v': 2.5478260869565217,
  'internvl2': 3.3550724637681157,
  'gpt_debate': 1.6492753623188405},
 'cvrr_non_existent_actions_with_non_existent_scene_depictions': {'gpt4v': 2.573611111111111,
  'internvl2': 3.020833333333333,
  'gpt_deb

In [12]:
for dataset, data in dataset2agreement.items():
    for judge in judge_model:
        if judge == 'gpt_debate':
            continue
        print(f'{dataset}-{judge}')
        print(dataset2agreement[dataset][judge]['average'])

cvrr_continuity_and_object_instance_count-gpt4v
0.21607264446552574
cvrr_continuity_and_object_instance_count-internvl2
0.08209556183784747
cvrr_fine_grained_action_understanding-gpt4v
0.3347103945940823
cvrr_fine_grained_action_understanding-internvl2
0.16283780069127285
cvrr_interpretation_of_social_context-gpt4v
0.4252708317714696
cvrr_interpretation_of_social_context-internvl2
0.10265816426503083
cvrr_interpretation_of_visual_context-gpt4v
0.3396661711507173
cvrr_interpretation_of_visual_context-internvl2
0.12017052673735265
cvrr_multiple_actions_in_a_single_video-gpt4v
0.3132240716480725
cvrr_multiple_actions_in_a_single_video-internvl2
0.12517084619812674
cvrr_non_existent_actions_with_existent_scene_depictions-gpt4v
0.13455850080130743
cvrr_non_existent_actions_with_existent_scene_depictions-internvl2
-0.04359798978596763
cvrr_non_existent_actions_with_non_existent_scene_depictions-gpt4v
0.027219198738512085
cvrr_non_existent_actions_with_non_existent_scene_depictions-internvl2


In [13]:
dataset2agreement[dataset]

{'gpt4v': {'llama_vid': 0.16522301767861247,
  'gpt4v': 0.07285596825510321,
  'video_chatgpt': 0.1896512851345259,
  'mplug_owl_Video': 0.2585898172235548,
  'video_llava': 0.18977313647821392,
  'average': 0.17521864495400205},
 'internvl2': {'llama_vid': 0.02007774881830615,
  'gpt4v': -0.02747523813138031,
  'video_chatgpt': 0.04339718596372244,
  'mplug_owl_Video': 0.04951543924353574,
  'video_llava': 0.03605289025588987,
  'average': 0.02431360523001478}}